<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#模型融合之Stacking-(判定贷款用户是否逾期)" data-toc-modified-id="模型融合之Stacking-(判定贷款用户是否逾期)-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>模型融合之Stacking (判定贷款用户是否逾期)</a></span><ul class="toc-item"><li><span><a href="#数据集划分" data-toc-modified-id="数据集划分-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>数据集划分</a></span></li><li><span><a href="#模型评估" data-toc-modified-id="模型评估-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>模型评估</a></span></li><li><span><a href="#模型融合---Stacking" data-toc-modified-id="模型融合---Stacking-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>模型融合 - Stacking</a></span><ul class="toc-item"><li><span><a href="#调包实现" data-toc-modified-id="调包实现-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>调包实现</a></span></li><li><span><a href="#自己实现" data-toc-modified-id="自己实现-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>自己实现</a></span></li></ul></li></ul></li><li><span><a href="#遇到的问题" data-toc-modified-id="遇到的问题-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>遇到的问题</a></span></li><li><span><a href="#Reference" data-toc-modified-id="Reference-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Reference</a></span></li></ul></div>

# 模型融合之Stacking (判定贷款用户是否逾期)

给定金融数据，预测贷款用户是否会逾期。
（status是标签：0表示未逾期，1表示逾期。）

**Task7（模型融合）** - 对Task6调优后的模型, 进行模型融合。例如, 用目前评分最高的模型作为基准模型, 和其他模型进行stacking模型融合, 得到最终模型和评分。

In [1]:
import pandas as pd

# 数据集预览
data = pd.read_csv('data.csv')
# 去重
data.drop_duplicates(inplace=True)
print(data.shape)
data.head()

(4754, 90)


,Unnamed: 0,custid,trade_no,bank_card_no,low_volume_percent,middle_volume_percent,take_amount_in_later_12_month_highest,trans_amount_increase_rate_lately,trans_activity_month,trans_activity_day,...,loans_max_limit,loans_avg_limit,consfin_credit_limit,consfin_credibility,consfin_org_count_current,consfin_product_count,consfin_max_limit,consfin_avg_limit,latest_query_day,loans_latest_day
0,5,2791858,20180507115231274000000023057383,卡号1,0.01,0.99,0,0.90,0.55,0.313,...,2900.0,1688.0,1200.0,75.0,1.0,2.0,1200.0,1200.0,12.0,18.0
1,10,534047,20180507121002192000000023073000,卡号1,0.02,0.94,2000,1.28,1.00,0.458,...,3500.0,1758.0,15100.0,80.0,5.0,6.0,22800.0,9360.0,4.0,2.0
2,12,2849787,20180507125159718000000023114911,卡号1,0.04,0.96,0,1.00,1.00,0.114,...,1600.0,1250.0,4200.0,87.0,1.0,1.0,4200.0,4200.0,2.0,6.0
3,13,1809708,20180507121358683000000388283484,卡号1,0.00,0.96,2000,0.13,0.57,0.777,...,3200.0,1541.0,16300.0,80.0,5.0,5.0,30000.0,12180.0,2.0,4.0
4,14,2499829,20180507115448545000000388205844,卡号1,0.01,0.99,0,0.46,1.00,0.175,...,2300.0,1630.0,8300.0,79.0,2.0,2.0,8400.0,8250.0,22.0,120.0


In [2]:
import pickle
# 载入特征
with open('feature.pkl', 'rb') as f:
    X = pickle.load(f)

# 观测正负样本是否均衡
y = data.status
y.value_counts()

0    3561
1    1193
Name: status, dtype: int64

## 数据集划分

In [3]:
# 划分训练集测试集
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=2333)

# 特征归一化
std = StandardScaler()
X_train = std.fit_transform(X_train.values)
X_test = std.transform(X_test.values)

## 模型评估

In [4]:
from sklearn.metrics import accuracy_score, roc_auc_score

def model_metrics(clf, X_train, X_test, y_train, y_test):
    # 预测
    y_train_pred = clf.predict(X_train)
    y_test_pred = clf.predict(X_test)
    
    y_train_proba = clf.predict_proba(X_train)[:,1]
    y_test_proba = clf.predict_proba(X_test)[:,1]
    
    # 准确率
    print('[准确率]', end = ' ')
    print('训练集：', '%.4f'%accuracy_score(y_train, y_train_pred), end = ' ')
    print('测试集：', '%.4f'%accuracy_score(y_test, y_test_pred))
    
    # auc取值：用roc_auc_score或auc
    print('[auc值]', end = ' ')
    print('训练集：', '%.4f'%roc_auc_score(y_train, y_train_proba), end = ' ')
    print('测试集：', '%.4f'%roc_auc_score(y_test, y_test_proba))

## 模型融合 - Stacking

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from xgboost.sklearn import XGBClassifier
from lightgbm.sklearn import LGBMClassifier

from mlxtend.classifier import StackingClassifier

In [6]:
# 模型调优后得到的参数
lr = LogisticRegression(C = 0.1, penalty = 'l1')
svm_linear = svm.SVC(C = 0.01, kernel = 'linear', probability=True)
svm_poly =  svm.SVC(C = 0.01, kernel = 'poly', probability=True)
svm_rbf =  svm.SVC(gamma = 0.01, C =0.01 , probability=True)
svm_sigmoid =  svm.SVC(C = 0.01, kernel = 'sigmoid',probability=True)
dt = DecisionTreeClassifier(max_depth=11,min_samples_split=550,min_samples_leaf=80,max_features=19)
xgb = XGBClassifier(learning_rate =0.01, n_estimators=180, max_depth=3, min_child_weight=5, 
                    gamma=0.4, subsample=0.5, colsample_bytree=0.9, reg_alpha=1, 
                    objective= 'binary:logistic', nthread=4,scale_pos_weight=1, seed=27)
lgb = LGBMClassifier(learning_rate =0.1, n_estimators=60, max_depth=3, min_child_weight=11, 
                    gamma=0.1, subsample=0.5, colsample_bytree=0.8, reg_alpha=1e-5, 
                    nthread=4,scale_pos_weight=1, seed=27)

### 调包实现

> 使用4种SVM、决策树、XGBoost和LightGBM作为初级分类器，LR作为次级分类器。

1）将初级分类器产生的类别标签作为新特征

In [7]:
sclf = StackingClassifier(classifiers=[svm_linear, svm_poly, svm_rbf, 
                                       svm_sigmoid, dt, xgb, lgb], meta_classifier=lr)

In [8]:
import warnings
warnings.filterwarnings("ignore")

sclf.fit(X_train, y_train.values)
model_metrics(sclf, X_train, X_test, y_train, y_test)

[准确率] 训练集： 0.8242 测试集： 0.8066
[auc值] 训练集： 0.6981 测试集： 0.6765


2）将初级分类器产生的输出类概率作为新特征

对输出概率use_probas=True，有两种不同的处理方式。

>假设有2个初级分类器和3个类别输出概率：p1=[0.2,0.5,0.3],p2=[0.3,0.4,0.4]。

>如果average_probas=True，则对分类器的结果求平均，得到：p=[0.25,0.45,0.35]

>[推荐]如果average_probas=False，则分类器的所有结果都保留作为新的特征：p=[0.2,0.5,0.3,0.3,0.4,0.4]

In [9]:
sclf = StackingClassifier(classifiers=[svm_linear, svm_poly, svm_rbf, svm_sigmoid, dt, xgb, lgb], 
                            meta_classifier=lr, use_probas=True,average_probas=False)

In [10]:
sclf.fit(X_train, y_train.values)
model_metrics(sclf, X_train, X_test, y_train, y_test)

[准确率] 训练集： 0.8365 测试集： 0.8017
[auc值] 训练集： 0.8773 测试集： 0.7962


### 自己实现

> 参见博客[模型融合之blending和stacking](https://www.cnblogs.com/HongjianChen/p/9706806.html)

# 遇到的问题

1）sclf.fit(X,y)时会报错

解决：注意X_train是array,而y_train是Series且索引不一定从0开始。此处我们取y_train.values，将y_train也转成array。

2）融合后没有基础模型效果好

# Reference

[详解stacking过程](https://blog.csdn.net/wstcjf/article/details/77989963)
[集成学习原理小结](https://www.cnblogs.com/pinard/p/6131423.html)
[集成学习stacking](https://blog.csdn.net/winycg/article/details/84032459)
[【机器学习】模型融合方法概述
贝尔塔](https://zhuanlan.zhihu.com/p/25836678)